In [23]:
config = """--dataset AFHQ --model ResNet6ImageNet --texture\
       --data_path /nas2/lait/5000_Dataset_reform/Image/AFHQ/afhq\
       --buffer_path /workspace/hyuns/src/Dataset_distillation/methods/tm/buffer_images/exp\
       --tsne_model /workspace/hyuns/src/Dataset_distillation/methods/tm/buffer_images/exp/AFHQ/ResNet6ImageNet/replay_buffer_9.pt"""

In [24]:
config = [s.split(' ') for s in config.split('\t')[0].split('--') if len(s)!=0]

In [25]:
config = [s[:2] for s in config]
config = [['--'+s[0],s[1]] for s in config]

In [34]:
for x in config:
    print("\""+x[0]+"\"",',',"\""+x[1]+"\"")

"--dataset" , "AFHQ"
"--model" , "ResNet6ImageNet"
"--texture" , ""
"--data_path" , "/nas2/lait/5000_Dataset_reform/Image/AFHQ/afhq"
"--buffer_path" , "/workspace/hyuns/src/Dataset_distillation/methods/tm/buffer_images/exp"
"--tsne_model" , "/workspace/hyuns/src/Dataset_distillation/methods/tm/buffer_images/exp/AFHQ/ResNet6ImageNet/replay_buffer_9.pt"


In [43]:
import os ,sys
sys.path.append('/workspace/hyuns/src/Dataset_distillation/methods/tm')
from networks import ResNet6ImageNet

ImportError: cannot import name 'ResNet6ImageNet' from 'networks' (/workspace/hyuns/src/Dataset_distillation/networks/__init__.py)

In [46]:
import torch
from torch import nn

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, norm='instancenorm'):
        super(BasicBlock, self).__init__()
        self.norm = norm
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.GroupNorm(planes, planes, affine=True) if self.norm == 'instancenorm' else nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.GroupNorm(planes, planes, affine=True) if self.norm == 'instancenorm' else nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.GroupNorm(self.expansion*planes, self.expansion*planes, affine=True) if self.norm == 'instancenorm' else nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNetImageNet(nn.Module):
    def __init__(self, block, num_blocks, channel=3, num_classes=10, norm='instancenorm'):
        super(ResNetImageNet, self).__init__()
        self.in_planes = 64
        self.norm = norm

        self.conv1 = nn.Conv2d(channel, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.GroupNorm(64, 64, affine=True) if self.norm == 'instancenorm' else nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, self.norm))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        # out = F.avg_pool2d(out, 4)
        # out = out.view(out.size(0), -1)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out


def ResNet6ImageNet(channel, num_classes):
    return ResNetImageNet(BasicBlock, [1,1,1,1], channel=channel, num_classes=num_classes)



In [49]:
model = ResNet6ImageNet(3,10)

In [55]:
for name, child in model.named_children():
    print(name[])

conv1
bn1
maxpool
layer1
layer2
layer3
layer4
avgpool
classifier


In [56]:
model.layer4

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): GroupNorm(512, 512, eps=1e-05, affine=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): GroupNorm(512, 512, eps=1e-05, affine=True)
    (shortcut): Sequential(
      (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): GroupNorm(512, 512, eps=1e-05, affine=True)
    )
  )
)

In [61]:
model.__class__

__main__.ResNetImageNet